In [1]:
# Under construction.

from IPython.display import clear_output
from data_generator import DataGenerator
from medium_generator import MediumGenerator, disk_func, cosine_func
from ngsolve import * 
from ngsolve.webgui import Draw
from numpy.linalg import lstsq
import multiprocessing
from joblib import Parallel, delayed
import numpy as np
import time

dg = DataGenerator(maxh = (0.05, 0.2))

medium = MediumGenerator(cosine_func) # MediumGenerator(disk_func)

background_params = [{"type": 0, "x": 0.0, "y": 0.0, "r": 0.5, "v": 0.0}]
background_permittivity = medium.generate(background_params)

# create a sample with 4 bumps. 

params  = [{"type": 0, "x": 0.6 ,"y": 0.4, "r": 0.3, "v": 1}, 
           {"type": 0, "x": 0.7, "y": -0.6, "r": 0.3, "v": 1},
           {"type": 0, "x": -0.8, "y": -0.3, "r": 0.3, "v": 1},
           {"type": 0, "x": -0.5, "y": 0.7, "r": 0.3, "v": 1},
           {"type": 0, "x": -0.6 ,"y": 0.2, "r": 0.3, "v": 1}, 
           {"type": 0, "x": -0.7, "y": -0.9, "r": 0.3, "v": 1},
           {"type": 0, "x": -0.1, "y": -0.3, "r": 0.3, "v": 1},
           {"type": 0, "x": -0.2, "y": 0.3, "r": 0.3, "v": 1}]

permittivity  = medium.generate(params)


Mesh generation took 0.6738098571076989 seconds


In [2]:
n_in_dir  = 32
n_out_dir = 8

A = Matrix(2 * n_in_dir * n_out_dir, dg.fes.ndof, complex=True)
b = Vector(2 * n_in_dir * n_out_dir, complex=True)

inc_direction_angles = np.linspace(0, 2 * np.pi, n_in_dir, endpoint=False)
out_direction_angles = np.linspace(0, 2 * np.pi, n_out_dir, endpoint=False) + np.random.random()*2*np.pi

freq = 0.5

def single_loop(i):        
    mat = []
    vec = []
    
    kx = 2 * pi * freq * cos(inc_direction_angles[i])
    ky = 2 * pi * freq * sin(inc_direction_angles[i])

    psi = CF((exp(1j * kx * x) * exp(1j * ky * y)))

    u_scat = dg.solve(kx, ky, permittivity)

    for j in range(n_out_dir):
        p_kx = 2 * pi * freq * cos(out_direction_angles[j])
        p_ky = 2 * pi * freq * sin(out_direction_angles[j])

        phi = CF((exp(1j * p_kx * x) * exp(1j * p_ky * y)))

        true_val = Integrate( (permittivity - background_permittivity) * (phi) *  (u_scat + psi), dg.mesh)

        test_func = dg.fes.TestFunction()

        linear_form = LinearForm(dg.fes)

        linear_form += test_func * (phi) *  (psi) * (IfPos((x)**2 + (y)**2 - (1.5) **2,
                0,
                1))  * dx

        linear_form.Assemble()
            
        mat.append(linear_form.vec.FV().NumPy())
        vec.append(true_val)

    return mat, vec

def run_code():
    start = time.time()
    
    with multiprocessing.Pool(48) as p:
        tasks = [p.apply_async(single_loop, [i]) for i in range(n_in_dir)]

        finished = {}
        # postprocessing
        while len(finished) != len(tasks):
            for i, task in enumerate(tasks):
                if task.ready():
                    finished[i] = task.get()
                    for j in range(n_out_dir):
                        index = i * n_out_dir + j
                        A.NumPy()[2 * index, :]     = finished[i][0][j].real
                        A.NumPy()[2 * index + 1, :] = finished[i][0][j].imag
                        b[2 * index]                = finished[i][1][j].real
                        b[2 * index + 1]            = finished[i][1][j].imag
                    
    end_time = time.time()

    print('parallel time: {}'.format(end_time - start))
    
    start = time.time()

    v = lstsq(A.NumPy(),  b.NumPy(), rcond=1e-2)[0]

    end_time = time.time()

    print('solving time: {}'.format(end_time - start))
    
    return v

In [3]:
v = run_code()

0
12345
6789

10
11
1312
15
142317
161920
182122
24
2625
30
29
28
31














27
parallel time: 14.223717212677002
solving time: 11.667215585708618


In [ ]:
permittivity_update =  GridFunction(dg.fes)
permittivity_update.vec.data = v

scene = Draw(permittivity_update, dg.mesh)
clear_output() # redraw the scene with a new height
scene.Draw(height="50vh")